<a href="https://colab.research.google.com/github/alicetech-dev/deepseekTTS/blob/main/deepseekTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download and run the Ollama Linux install script
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# Get Ngrok authentication token from colab secrets environment
#obten tu token de Ngrok y agregalo a tu secrets de google colab con el nombre NGROK_AUT_TOKEN
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
# Install:
#  1. aiohttp for concurrent subprocess execution in Jupyter Notebooks
#  2. pyngrok for Ngrok wrapper
!pip install aiohttp pyngrok

import asyncio
import os

# Set LD_LIBRARY_PATH so the system NVIDIA library becomes preferred
# over the built-in library. This is particularly important for
# Google Colab which installs older drivers
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

# Define run - a helper function to run subcommands asynchronously.
# The function takes in 2 arguments:
#  1. command
#  2. environment variable
async def run(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE
  )


# This function is designed to handle large amounts of text data efficiently.
# It asynchronously iterate over lines and print them, stripping and decoding as needed.
  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))


# Gather the standard output (stdout) and standard error output (stderr) streams of a subprocess and pipe them through
# the `pipe()` function to print each line after stripping whitespace and decoding UTF-8.
# This allows us to capture and process both the standard output and error messages from the subprocess concurrently.
  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )


# Authenticate with Ngrok
await asyncio.gather(
  run(['ngrok', 'config', 'add-authtoken', NGROK_AUTH_TOKEN])
)

>>> starting ngrok config add-authtoken 2sMBQ87UvsxOFUdrhamt8RPRTfL_4PU5WQrifsQDbxbfnsmJR
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


[None]

In [ ]:
# Run multiple tasks concurrently:
#  1. Start the Ollama server.
#  2. Start ngrok to forward HTTP traffic from the local ollama api running on localhost:11434.
#     Instructions come from Ollama doc: https://github.com/ollama/ollama/blob/main/docs/faq.md#how-can-i-use-ollama-with-ngrok
await asyncio.gather(
    run(['ollama', 'serve']),

    # If you don't want to map to a static URL in Ngrok, uncomment line 9 and comment line 10 before running this cell
    # run(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434']),
    run(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434', '--domain', 'reemplazacontudomainde.ngrok-free.app']),
)

>>> starting ollama serve
>>> starting ngrok http --log stderr 11434 --host-header localhost:11434 --domain moccasin-just-pipefish.ngrok-free.app
t=2025-01-31T00:31:29+0000 lvl=info msg="no configuration paths supplied"
t=2025-01-31T00:31:29+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2025-01-31T00:31:29+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIMjH6XCPqbW80weESHsZBMdEKZrziA9EHu3NywvfVDVp

t=2025-01-31T00:31:29+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
2025/01/31 00:31:29 routes.go:1187: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http: